## Esercizio 12.03.25

*Introduzione*

Lo scenario presenta un test medico utilizzato per prevedere la probabilità di anomalie cromosomiche in donne in gravidanza over 35. Il test ha un'alta sensibilità (90%) ma una bassa precisione, portando a un alto numero di falsi positivi e un F1-score di 0,15. La sfida è creare un dataset sintetico che rispecchi questo contesto, adattare un modello di classificazione e confrontare i suoi risultati con il test descritto.

*Comprendere il Problema*

*Punti chiave:*

Prevalenza: 50 su 10.000 persone hanno la malattia.

Sensibilità (Recall): 90% - Il test identifica correttamente il 90% delle persone che svilupperanno la malattia.

Tasso di Falsi Positivi: 5% - Su 9.950 persone sane, 497 vengono erroneamente identificate come positive.

Precisione: 8% - Solo l'8% dei risultati positivi del test sono veri positivi.

F1-score: 0,15 - Una metrica che bilancia precisione e sensibilità.

*Obiettivo:*

-Creare un dataset sintetico con caratteristiche simili.

-Adattare un modello di classificazione.

-Confrontare le prestazioni del modello con il test descritto.

In [1]:
# Importazione delle librerie necessarie
import numpy as np
import pandas as pd
from sklearn.datasets import make_classification
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, confusion_matrix

# 1. Creazione del set di dati sintetico
# Parametri
n_samples = 10000  # Numero totale di campioni
prevalence = 0.005  # Prevalenza della malattia (0.5%)
sensitivity = 0.90  # Sensibilità (recall) del test
false_positive_rate = 0.05  # Tasso di falsi positivi del test

# Creazione di un set di dati sintetico
X, y = make_classification(
    n_samples=n_samples,
    n_features=10,  # Numero di caratteristiche (parametri misurati)
    n_informative=5,  # Numero di caratteristiche informative
    n_redundant=2,  # Numero di caratteristiche ridondanti
    n_clusters_per_class=1,  # Numero di cluster per classe
    weights=[1 - prevalence],  # Peso per la classe negativa (sani)
    flip_y=0,  # Rumore nei dati (0 per nessun rumore)
    random_state=42  # Seed per riproducibilità
)

# Conversione in un DataFrame per una migliore gestione
df = pd.DataFrame(X, columns=[f'feature_{i}' for i in range(X.shape[1])])
df['target'] = y

# 2. Divisione del set di dati in training e test set
X_train, X_test, y_train, y_test = train_test_split(
    df.drop('target', axis=1),  # Features
    df['target'],  # Target
    test_size=0.3,  # 30% dei dati per il test set
    random_state=42  # Seed per riproducibilità
)

# 3. Applicazione di SMOTE per bilanciare il set di dati
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

# 4. Addestramento del modello di classificazione (Random Forest)
model = RandomForestClassifier(random_state=42)
model.fit(X_train_smote, y_train_smote)

# 5. Predizione sul test set
y_pred = model.predict(X_test)

# 6. Valutazione del modello
print("Risultati del modello:")
print(classification_report(y_test, y_pred))
print("Matrice di confusione:")
print(confusion_matrix(y_test, y_pred))

# Calcolo dell'F1-Score
new_f1_score = f1_score(y_test, y_pred)
print(f"F1-Score del Nuovo Modello: {new_f1_score}")

# 7. Confronto con i risultati del test descritto
original_f1_score = 0.15  # F1-Score del test originale
print(f"F1-Score Originale: {original_f1_score}")

if new_f1_score > original_f1_score:
    print("Il nuovo modello ha un F1-Score migliore rispetto al test originale.")
else:
    print("Il nuovo modello non ha migliorato l'F1-Score rispetto al test originale.")

Risultati del modello:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2985
           1       0.86      0.40      0.55        15

    accuracy                           1.00      3000
   macro avg       0.93      0.70      0.77      3000
weighted avg       1.00      1.00      1.00      3000

Matrice di confusione:
[[2984    1]
 [   9    6]]
F1-Score del Nuovo Modello: 0.5454545454545454
F1-Score Originale: 0.15
Il nuovo modello ha un F1-Score migliore rispetto al test originale.


I risultati ottenuti dal modello di classificazione sono interessanti e mostrano un miglioramento significativo rispetto al test originale. Ecco un'analisi dettagliata dei risultati:
   
*Precisione (Classe 1): 0.86*

Questo valore indica che l'86% delle predizioni positive del modello sono corrette. È un valore molto alto, il che significa che il modello è molto preciso nel identificare i veri positivi.

*Recall (Classe 1): 0.40*

Il recall (o sensibilità) del 40% indica che il modello identifica correttamente il 40% dei casi positivi. Questo è inferiore alla sensibilità del test originale (90%), ma è importante notare che il test originale aveva un alto tasso di falsi positivi, che il modello sembra aver ridotto.
   
*F1-Score: 0.545*

L'F1-Score è una media armonica tra precisione e recall. Il valore di 0.545 è significativamente migliore rispetto all'F1-Score originale di 0.15. Questo indica che il modello bilancia meglio precisione e recall rispetto al test originale.

Matrice di Confusione:
   
Veri Negativi (True Negatives, TN): 2984

Il modello ha correttamente identificato 2984 casi negativi.

Falsi Positivi (False Positives, FP): 1

Solo 1 caso è stato erroneamente identificato come positivo. Questo è un miglioramento significativo rispetto al tasso di falsi positivi del 5% del test originale.

Falsi Negativi (False Negatives, FN): 9

9 casi positivi sono stati erroneamente identificati come negativi. Questo è un po' alto, ma è un compromesso accettabile dato il miglioramento nella precisione.

Veri Positivi (True Positives, TP): 6

Il modello ha correttamente identificato 6 casi positivi.

Considerazioni Finali:

*Miglioramento della Precisione*: Il modello ha una precisione molto alta (86%), il che significa che quando predice un caso positivo, è molto probabile che sia corretto. Questo è un miglioramento significativo rispetto al test originale, che aveva una precisione dell'8%.

*Compromesso sul Recall*: Il recall del 40% è inferiore alla sensibilità del test originale (90%), ma è un compromesso accettabile dato il miglioramento nella precisione e la riduzione dei falsi positivi.

*Riduzione dei Falsi Positivi*: Il modello ha solo 1 falso positivo, il che è un miglioramento enorme rispetto ai 497 falsi positivi del test originale.

Conclusioni:

*Il nuovo modello ha un F1-Score significativamente migliore rispetto al test originale, il che indica che è più efficace nel bilanciare precisione e recall.*

*Il modello ha una precisione molto alta e un basso tasso di falsi positivi, il che lo rende più affidabile per identificare i casi positivi.*

*Il recall è inferiore rispetto al test originale, ma questo è un compromesso accettabile dato il miglioramento complessivo delle prestazioni del modello.*

*In sintesi, il nuovo modello rappresenta un miglioramento significativo rispetto al test originale, specialmente in termini di precisione e riduzione dei falsi positivi. Tuttavia, potrebbe essere utile esplorare ulteriori tecniche per migliorare il recall senza compromettere la precisione.*